In [1]:
import pandas as pd

df = pd.read_csv("./data/train.csv").set_index("PassengerId")

df.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [2]:
from sklearn.model_selection import train_test_split

X = df.loc[:,df.columns != "Survived"]
y = df["Survived"]
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [3]:
# https://autokeras.com/tutorial/structured_data_classification/
import autokeras as ak

# Initialize the structured data classifier.
clf = ak.StructuredDataClassifier(
    overwrite=True, # When building of model fails, 
                     # set to False and then back to True
    max_trials=5) # It tries 5 different models.

# Feed the structured data classifier with training data.
clf.fit(x_train,
        y_train,
        # Split the training data and use the last 15% as validation data.
        validation_split=0.15,
        epochs=50)

# returns scalar test loss (if the model has a single output and no metrics)
# or list of scalars (if the model has multiple outputs and/or metrics)
print(clf.evaluate(x_test, y_test))

Trial 5 Complete [00h 00m 04s]
val_accuracy: 0.8888888955116272

Best val_accuracy So Far: 0.9027777910232544
Total elapsed time: 00h 00m 20s
INFO:tensorflow:Oracle triggered exit
Epoch 1/50
23/23 [==============================] - 1s 2ms/step - loss: 0.6593 - accuracy: 0.7111
Epoch 2/50
23/23 [==============================] - 0s 3ms/step - loss: 0.3971 - accuracy: 0.8265
Epoch 3/50
23/23 [==============================] - 0s 3ms/step - loss: 0.3419 - accuracy: 0.8376
Epoch 4/50
23/23 [==============================] - 0s 3ms/step - loss: 0.2843 - accuracy: 0.8808
Epoch 5/50
23/23 [==============================] - 0s 3ms/step - loss: 0.2536 - accuracy: 0.8961
Epoch 6/50
23/23 [==============================] - 0s 3ms/step - loss: 0.2307 - accuracy: 0.9109
Epoch 7/50
23/23 [==============================] - 0s 3ms/step - loss: 0.2104 - accuracy: 0.9278
Epoch 8/50
23/23 [==============================] - 0s 3ms/step - loss: 0.1927 - accuracy: 0.9345
Epoch 9/50
23/23 [==================

6/6 [==============================] - 0s 1ms/step - loss: 1.4201 - accuracy: 0.6480
[1.4201401472091675, 0.6480447053909302]


In [4]:
model = clf.export_model()
model.summary()
# labels for the scalar outputs
print(model.metrics_names)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 10)]              0         
_________________________________________________________________
multi_category_encoding (Mul (None, 10)                0         
_________________________________________________________________
normalization (Normalization (None, 10)                21        
_________________________________________________________________
dense (Dense)                (None, 512)               5632      
_________________________________________________________________
batch_normalization (BatchNo (None, 512)               2048      
_________________________________________________________________
re_lu (ReLU)                 (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               262656

In [5]:
import numpy as np
print(model.evaluate(x_test.astype(np.unicode), y_test))

6/6 [==============================] - 0s 2ms/step - loss: 1.4201 - accuracy: 0.6480
[1.4201401472091675, 0.6480447053909302]


In [6]:
import tensorflow as tf

try:
    model.save("model_autokeras", save_format="tf")
except:
    model.save("model_autokeras.h5")

INFO:tensorflow:Assets written to: model_autokeras/assets


In [7]:
import json 

row = x_test.iloc[0]
json_row = row.to_json(orient="index")
json_row

'{"Pclass":3,"Name":"Moubarek, Master. Halim Gonios (\\"William George\\")","Sex":"male","Age":null,"SibSp":1,"Parch":1,"Ticket":"2661","Fare":15.2458,"Cabin":null,"Embarked":"C"}'

In [8]:
rows = x_test.iloc[0:5]
labels = y_test.iloc[0:5]
json_rows = rows.to_json(orient="records")
json_rows

'[{"Pclass":3,"Name":"Moubarek, Master. Halim Gonios (\\"William George\\")","Sex":"male","Age":null,"SibSp":1,"Parch":1,"Ticket":"2661","Fare":15.2458,"Cabin":null,"Embarked":"C"},{"Pclass":2,"Name":"Kvillner, Mr. Johan Henrik Johannesson","Sex":"male","Age":31.0,"SibSp":0,"Parch":0,"Ticket":"C.A. 18723","Fare":10.5,"Cabin":null,"Embarked":"S"},{"Pclass":3,"Name":"Alhomaki, Mr. Ilmari Rudolf","Sex":"male","Age":20.0,"SibSp":0,"Parch":0,"Ticket":"SOTON\\/O2 3101287","Fare":7.925,"Cabin":null,"Embarked":"S"},{"Pclass":2,"Name":"Harper, Miss. Annie Jessie \\"Nina\\"","Sex":"female","Age":6.0,"SibSp":0,"Parch":1,"Ticket":"248727","Fare":33.0,"Cabin":null,"Embarked":"S"},{"Pclass":3,"Name":"Nicola-Yarred, Miss. Jamila","Sex":"female","Age":14.0,"SibSp":1,"Parch":0,"Ticket":"2651","Fare":11.2417,"Cabin":null,"Embarked":"C"}]'

In [9]:
labels

PassengerId
710    1
440    0
841    0
721    1
40     1
Name: Survived, dtype: int64

In [10]:
record = pd.read_json(json_row, orient="index")
records = pd.read_json(json_rows)
record

,0
Pclass,3
Name,"Moubarek, Master. Halim Gonios (""William George"")"
Sex,male
Age,None
SibSp,1
Parch,1
Ticket,2661
Fare,15.2458
Cabin,None
Embarked,C


In [11]:
records

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Moubarek, Master. Halim Gonios (""William George"")",male,NaN,1,1,2661,15.2458,NaN,C
1,2,"Kvillner, Mr. Johan Henrik Johannesson",male,31.0,0,0,C.A. 18723,10.5000,NaN,S
2,3,"Alhomaki, Mr. Ilmari Rudolf",male,20.0,0,0,SOTON/O2 3101287,7.9250,NaN,S
3,2,"Harper, Miss. Annie Jessie ""Nina""",female,6.0,0,1,248727,33.0000,NaN,S
4,3,"Nicola-Yarred, Miss. Jamila",female,14.0,1,0,2651,11.2417,NaN,C


In [12]:
rows.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5 entries, 710 to 40
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    5 non-null      int64  
 1   Name      5 non-null      object 
 2   Sex       5 non-null      object 
 3   Age       4 non-null      float64
 4   SibSp     5 non-null      int64  
 5   Parch     5 non-null      int64  
 6   Ticket    5 non-null      object 
 7   Fare      5 non-null      float64
 8   Cabin     0 non-null      object 
 9   Embarked  5 non-null      object 
dtypes: float64(2), int64(3), object(5)
memory usage: 612.0+ bytes


In [13]:
x_test_unicode = x_test.astype(np.unicode)
x_test_unicode

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,
710,3,"Moubarek, Master. Halim Gonios (""William George"")",male,nan,1,1,2661,15.2458,nan,C
440,2,"Kvillner, Mr. Johan Henrik Johannesson",male,31.0,0,0,C.A. 18723,10.5,nan,S
841,3,"Alhomaki, Mr. Ilmari Rudolf",male,20.0,0,0,SOTON/O2 3101287,7.925,nan,S
721,2,"Harper, Miss. Annie Jessie ""Nina""",female,6.0,0,1,248727,33.0,nan,S
40,3,"Nicola-Yarred, Miss. Jamila",female,14.0,1,0,2651,11.2417,nan,C
...,...,...,...,...,...,...,...,...,...,...
434,3,"Kallio, Mr. Nikolai Erland",male,17.0,0,0,STON/O 2. 3101274,7.125,nan,S
774,3,"Elias, Mr. Dibo",male,nan,0,0,2674,7.225,nan,C
26,3,"Asplund, Mrs. Carl Oscar (Selma Augusta Emilia...",female,38.0,1,5,347077,31.3875,nan,S


In [14]:
from tensorflow.keras.models import load_model

loaded_model = load_model("model_autokeras", 
                          custom_objects=ak.CUSTOM_OBJECTS)
print(loaded_model.evaluate(x_test_unicode, y_test))

6/6 [==============================] - 0s 1ms/step - loss: 1.4201 - accuracy: 0.6480
[1.4201401472091675, 0.6480447053909302]


In [15]:
predictions = loaded_model.predict(rows.astype(np.unicode))
print(predictions)
print(predictions.T[0])

[[0.0027537 ]
 [0.19685769]
 [0.64665985]
 [0.99973816]
 [0.6687872 ]]
[0.0027537  0.19685769 0.64665985 0.99973816 0.6687872 ]


In [16]:
labels

PassengerId
710    1
440    0
841    0
721    1
40     1
Name: Survived, dtype: int64

In [17]:
print(loaded_model.predict(rows.astype(np.unicode)))

[[0.0027537 ]
 [0.19685769]
 [0.64665985]
 [0.99973816]
 [0.6687872 ]]
